In [1]:
# Please do not change this cell because some hidden tests might depend on it.
import os

# Otter grader does not handle ! commands well, so we define and use our
# own function to execute shell commands.
def shell(commands, warn=True):
    """Executes the string `commands` as a sequence of shell commands.

       Prints the result to stdout and returns the exit status.
       Provides a printed warning on non-zero exit status unless `warn`
       flag is unset.
    """
    file = os.popen(commands)
    print (file.read().rstrip('\n'))
    exit_status = file.close()
    if warn and exit_status != None:
        print(f"Completed with errors. Exit status: {exit_status}\n")
    return exit_status

shell("""
ls requirements.txt >/dev/null 2>&1
if [ ! $? = 0 ]; then
 rm -rf .tmp
 git clone https://github.com/cs236299-2024-winter/lab1-1.git .tmp
 mv .tmp/tests ./
 mv .tmp/requirements.txt ./
 rm -rf .tmp
fi
pip install -q -r requirements.txt
""")

In [2]:
# Initialize Otter
import otter
grader = otter.Notebook()

# Course 236299
## Lab 1-1 – Types, tokens, and representing text

In [3]:
import math
import re
import sys

import torch
import nltk

In [4]:
nltk.download('punkt', quiet=True) # this module is used to tokenize the text

True

Where we're headed: Nearest neighbor text classification works by classifying a novel text with the same class as that of the training text that is closest according to some distance metric. These metrics are calculated based on representations of the texts. In this lab, we'll introduce some different representations and you'll use nearest neighbor classification to predict the speaker of sentences selected from a children's book.
    
The objectives of this lab are to:

* Clarify terminology around words and texts,
* Manipulate different representations of words and texts,
* Apply these representations to calculate text similarity, and
* Classify documents by a simple nearest neighbor model.
   
Recall that in this and all labs, we will have you carry out several exercises in notebook cells. The cells you are to do are marked '`#TODO`'. They will typically have a `...` where your code or answer should go. Where specified, you needn't write code to calculate the answer, but instead, simply work out the answer yourself and enter it.

New bits of Python used for the first time in the _solution set_ for this lab, and which you may therefore find useful:

* `math.acos`
* `math.pi`
* `re.match`
* `set`
* `sorted`
* `str.join`
* `str.lower`
* `torch.amax`
* `torch.dot`
* `torch.linalg.norm`
* `torch.maximum`
* `torch.minimum`
* `torch.stack`
* `torch.sum`
* `torch.Tensor.type`
* `torch.where`
* `torch.zeros`
* `torch.zeros_like`


# Counting words

<img src="https://github.com/nlp-236299/data/blob/master/Seuss/seuss%20-%201966%20-%20green%20eggs%20and%20ham.gif?raw=true" width=150 align=right />

Here are five sentences from Dr. Seuss's [_Green Eggs and Ham_](https://en.wikipedia.org/wiki/Green_Eggs_and_Ham):

    Would you like them here or there?
    I would not like them here or there.
    I would not like them anywhere.
    I do not like green eggs and ham.
    I do not like them, Sam-I-am.

Let's make this text available in the variable `text`.

In [5]:
text = """
    Would you like them here or there?
    I would not like them here or there.
    I would not like them anywhere.
    I do not like green eggs and ham.
    I do not like them, Sam-I-am.
    """

A Python string like this is, of course, a sequence of characters. But we think of this text as a sequence of sentences each composed of a sequence of words. How many words are there in this text? That is a fraught question, for several reasons, including

* The type-token distinction
* Tokenization issues
* Normalization

## Types versus tokens

In determining the number of words in `text`, are we talking about word _types_ or word _tokens_. (For instance, there are five _tokens_ of the word _type_ 'like' in the example `text`.)

How many word tokens are there in total in this text? (Just count them manually.) Assign the number to the variable `token_count` in the next cell.
<!--
BEGIN QUESTION
name: token_count
-->

In [6]:
#TODO - define `token_count` to be the number of tokens in `text`
token_count = 35

In [7]:
grader.check("token_count")

All tests passed!

How many word types are there? (Again, you can just count manually.)
<!--
BEGIN QUESTION
name: type_count
-->

In [8]:
#TODO - define `type_count` to be the number of types in `text`
type_count = 21

In [9]:
grader.check("type_count")

All tests passed!

<!-- BEGIN QUESTION -->

**Question:** The set of types of a language is referred to as its _vocabulary_. Are there more types or tokens as you calculated above? Could it be otherwise?
<!--
BEGIN QUESTION
name: type_vs_token_count
manual: true
-->

Tokens are instances of word types, which means that there is at least one token per each type, hence the number of tokens is always equal or greater than the number of type.

<!-- END QUESTION -->



## Tokenization

Did you count 'there?' as one token or two? This raises the issue of _tokenization_ of text, how to decide where the token boundaries occur. For instance, here's a simple way to split a string – to _tokenize_ it – in Python by splitting at whitespace.

In [10]:
def whitespace_tokenize(str):
    return str.split()

Try it out on the `text` defined above.
<!--
BEGIN QUESTION
name: tokens_whitespace
-->

In [11]:
#TODO - define `tokens` to be the tokens as defined by the `whitespace_tokenize` function
tokens = whitespace_tokenize(text)

In [12]:
grader.check("tokens_whitespace")

All tests passed!

Using this tokenization method, count the number of tokens in the text, this time using Python to do the work.
<!--
BEGIN QUESTION
name: token_count_whitespace
-->

In [13]:
#TODO - place your token count here
token_count_2 = len(tokens)

In [14]:
grader.check("token_count_whitespace")

All tests passed!

Arguably, we _should_ split off punctuation as separate tokens, but even there, some care must be taken. We don't want to split 'don't' into three tokens or 'Sam-I-am' into five. (There's a good argument to be made however that the string 'don't' should be construed as two tokens, namely, 'do' and 'n't', but that's beyond the scope of today's discussion.)

Here, we provide an alternative tokenizer that splits tokens at whitespace and splits off punctuation at the beginning and end of non-whitespace regions as separate tokens as well. It makes use of [the Python `re` module](https://docs.python.org/3/library/re.html) for regular expressions to specify the splitting process. Look over the code and make sure you understand what's going on. You might find [this online tool](https://regexr.com/) useful.

In [15]:
def punc_tokenize(str):
    return [tok for tok in re.split('(\W*?)\s+', str) if tok != '']

Now how many tokens are there in the text if tokenized in this way?
<!--
BEGIN QUESTION
name: token_count_punc
-->

In [16]:
#TODO
token_count_3 = len(punc_tokenize(text))

In [17]:
grader.check("token_count_punc")

All tests passed!

## Normalization

This tokenization method counts 'Would' and 'would' (capitalized and uncapitalized) as separate types. Is that a good idea? This raises the issue of text _normalization_.

Define a function `normalize_token` that normalizes tokens by making them lowercase if at most the first character is uppercase. (Hints [here](https://docs.python.org/3/library/stdtypes.html#str.lower) and [here](https://docs.python.org/3/library/re.html#re.match). These are also listed in the hint cell at the top of the lab, so we'll mostly stop providing these hints from here on.)
<!--
BEGIN QUESTION
name: normalize_token
-->

In [18]:
#TODO - implement normalize_token, which returns the normalized word for a single word `str`
def normalize_token(str):
    if re.match(r'^[A-Z]', str):
        return str.lower()
    else:
        return str

In [19]:
grader.check("normalize_token")

All tests passed!

Now define `text_tokenized` to be the sequence of normalized tokens as tokenized by `punc_tokenize`
<!--
BEGIN QUESTION
name: text_tokenized
-->

In [20]:
#TODO
text_tokenized = [normalize_token(token) for token in punc_tokenize(text)]

In [21]:
grader.check("text_tokenized")

All tests passed!

How many types are there when tokenized and normalized in this way?
<!--
BEGIN QUESTION
name: type_count_norm_punc
-->

In [22]:
#TODO
type_count_norm_punc = len(set(text_tokenized))

In [23]:
grader.check("type_count_norm_punc")

All tests passed!

## Using prebuilt tokenizers

Tokenization and normalization are so commonly needed that many packages provide pre-built tokenizers of various sorts. We'll use one from the [Natural Language Tool Kit (NLTK)](http://nltk.org). The package has already been imported above under the name `nltk`.

Define two tokenizers, versions of `whitespace_tokenize` and a normalized version of `punc_tokenize` above, using [the `nltk.tokenize.WhitespaceTokenizer`](https://www.nltk.org/api/nltk.tokenize.regexp.html#nltk.tokenize.regexp.WhitespaceTokenizer) and [`nltk.tokenize.word_tokenize`](https://www.nltk.org/api/nltk.tokenize.html#module-nltk.tokenize) respectively.
Note that `nltk.tokenize.word_tokenize` only tokenizes the string, so we normalize the string by lowering the string's characters.
<!--
BEGIN QUESTION
name: nltk_whitespace_tokenize_and_nltk_normpunc_tokenize
-->

In [24]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer

def nltk_whitespace_tokenize(str):
    return nltk.tokenize.WhitespaceTokenizer().tokenize(str)

def nltk_normpunc_tokenize(str):
  return word_tokenize(str.lower())

In [25]:
grader.check("nltk_whitespace_tokenize_and_nltk_normpunc_tokenize")

All tests passed!

We'll print out the last few tokens of the `text` tokenized by the whitespace tokenizer and the NLTK tokenizer to see the differences.

In [26]:
print(nltk_whitespace_tokenize(text)[-10:])
print(nltk_normpunc_tokenize(text)[-10:])

['green', 'eggs', 'and', 'ham.', 'I', 'do', 'not', 'like', 'them,', 'Sam-I-am.']
['ham', '.', 'i', 'do', 'not', 'like', 'them', ',', 'sam-i-am', '.']


> _Meta-comment:_ Because it's important that you get practice both with implementing the ideas in the course from first principles and also with using prebuilt software that provides similar functionality, we'll often have you engage in this seemingly redundant process of first implementing a small example and then applying a prebuilt method to do much the same thing. The effort may be duplicative, but it is not wasted.

In the next section, it will be helpful to have the tokenized text available. Define `nltk_text_tokenized` to be the sequence of normalized tokens of the sample `text` as tokenized by the tokenizer `nltk_normpunc_tokenize` that you've just written.
<!--
BEGIN QUESTION
name: nltk_text_tokenized
-->

In [27]:
#TODO
nltk_text_tokenized = nltk_normpunc_tokenize(text)

In [28]:
grader.check("nltk_text_tokenized")

All tests passed!

> _Meta-comment:_ Chooing which tokenization method to use is an important (and unsolved!) question even in the age of large language models (LLMs). For more information, see ["Tokenizer Choice For LLM Training: Negligible or Crucial?"](https://arxiv.org/pdf/2310.08754).

# Representing words

In this section, we'll explore some simple representations for tokens, as a step on the way to representing texts – sentences or documents:

## String encoding
We've already seen string encoding above, representing a token of a word type by a string specific to that type: a token 'green' represented by an instance of the Python string `'green'`, for instance, or 'Sam-I-am' represented by `'Sam-I-am'`. So let's move on.

## 1-hot encoding
Given a vocabulary for a language, we can associate each type with an integer, say by its index in a vector. We've already imported the `torch` module; we'll use `torch` tensors for the index vector. For the Seuss text, we can use the following list named `vocabulary` to represent the ordered vocabulary:

In [29]:
vocabulary = sorted(set(nltk_text_tokenized))
vocabulary

[',',
 '.',
 '?',
 'and',
 'anywhere',
 'do',
 'eggs',
 'green',
 'ham',
 'here',
 'i',
 'like',
 'not',
 'or',
 'sam-i-am',
 'them',
 'there',
 'would',
 'you']

### A digression on `torch` tensors

Recall that `torch` tensors allow for vectorized computations: many operations on them work [*componentwise*](https://en.wikipedia.org/wiki/Pointwise#Componentwise_operations), that is, separately for each component of the tensor, rather than on the tensor all at once. Compare the following two operations, first on lists, then on `torch` tensors.

In [30]:
[1, 2] == 1

False

In [31]:
torch.tensor([1, 2]) == 1

tensor([ True, False])

This behavior of tensors is quite powerful, allowing for simply specifying complex operations and for efficient, even parallelizable, computation of them. You'll want ot take advantage of these characteristics of tensors where possible, here and in future assignments.

But back to the 1-hot representation.

In the _1-hot representation_ of words, a token is then represented by a bit vector (again given as a `torch` tensor), with a 1 at the index of the token's type. (For consistency with some later `torch` functions, we'll take the elements to be floats rather than ints. The `.type` method is useful for converting the type, and it even conveniently broadcasts componentwise over the tensors.) For instance, the 1-hot representation of the comma token ',' would be

In [32]:
torch.tensor([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]).type(torch.float32)

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.])

Conversion back and forth between these various representations is useful. Define functions `str_to_onehot` and `onehot_to_str` that convert between the string and one-hot representations using a vocabulary array to define the  conversion.

Ideally, in your implementation, you'll want to take advantage of the componentwise nature of many tensor operations discussed above.

In [33]:
#TODO
def str_to_onehot(wordtype, vocab):
    """Returns the 1-hot representation of `wordtype` with vocabulary
    `vocab`.
    The returned value should be a `torch.tensor` with data type `float`.
    """
    idx = vocab.index(wordtype)
    onehot = torch.zeros(len(vocab)).type(torch.float32)
    onehot[idx] = 1
    return onehot

def onehot_to_str(onehot, vocab):
    """Returns the string representation of `onehot`, a one-hot
    representation of a word type, with vocabulary `vocab`.
    """
    return vocab[torch.argmax(onehot)]

Now use `str_to_onehot` to define the variable `anywhere_1hot` to be the 1-hot representation for a token of the type 'anywhere'.
<!--
BEGIN QUESTION
name: anywhere_1hot
-->

In [34]:
#TODO
anywhere_1hot = str_to_onehot("anywhere", vocabulary)

In [35]:
grader.check("anywhere_1hot")

All tests passed!

You can verify that the conversion worked correctly by inverting it using `onehot_to_str`, which we've done in the following unit test.
<!--
BEGIN QUESTION
name: anywhere_1hot_reverse
-->

In [36]:
grader.check("anywhere_1hot_reverse")

All tests passed!

# Representing texts

## The set-of-words representation

We can represent a whole *text* (a sequence of words) by manipulating the vector representations of the words within the text. For instance, we can take the componentwise maximum of the vectors. We refer to this as the _set-of-words_ representation.

Here we've defined a function `set_of_words` that returns the set of words representation for a token sequence.

In [37]:
def set_of_words(tokens, vocabulary):
    """Returns the set-of-words representation as a tensor of floats for the
    sequence of `tokens` using the `vocabulary` to specify the conversion.
    """
    onehots = torch.stack([str_to_onehot(token, vocabulary) for token in tokens])
    return torch.amax(onehots, 0).type(torch.float32)

The set-of-words representation for a text is a vector that has a `1` for each word type that occurs in the text. The vector represents the [characteristic function](https://en.wikipedia.org/wiki/Characteristic_function) for the subset of vocabulary words that appear in the text; hence the term 'set of words'.

What is the set-of-words representation for the example text 'I would not, would not, here or there.'?

Define the variable `example_sow` to be the set of words representation for the example text 'I would not, would not, here or there.' Use the `nltk_normpunc_tokenize` tokenizer.

<!--
BEGIN QUESTION
name: example_sow
-->

In [38]:
#TODO
example_sow = set_of_words(nltk_normpunc_tokenize("I would not, would not, here or there."), vocabulary)

In [39]:
grader.check("example_sow")

All tests passed!

## The bag of words representation

If instead, we take the componentwise _sum_ of the vectors instead of the maximum, the text representation provides the _frequency_ of each word type in the text. We refer to this representation as the _[bag](https://en.wikipedia.org/wiki/Multiset) of words_ representation.

Define a function `bag_of_words`, analogous to `set_of_words` above, that returns the bag-of-words representation for a token sequence.

In [40]:
#TODO
def bag_of_words(tokens, vocabulary):
    onehots = torch.stack([str_to_onehot(token, vocabulary) for token in tokens])
    return torch.sum(onehots, 0).type(torch.float32)

What is the bag of words representation for the example text 'I would not, would not, here or there.'?
<!--
BEGIN QUESTION
name: example_bow
-->

In [41]:
#TODO - define the variable to be the bag of words representation for the example text
# 'I would not, would not, here or there.'
# Use the `nltk_normpunc_tokenize` tokenizer
example_bow = bag_of_words(nltk_normpunc_tokenize("I would not, would not, here or there."), vocabulary)

In [42]:
grader.check("example_bow")

All tests passed!

# Document similarity metrics

Consider the following text classification problem: Each sentence in _Green Eggs and Ham_ is spoken by one of two characters, Sam-I-am and Guy-am-I. We want to be able to classify new sentences as (most likely) being uttered by one of the two.

A simple method for text classification is the _nearest neighbor_ method. We select the class for the new sentence that is the same as the class of the "nearest" (most similar) sentence for which we already know the class. (You'll experiment much more with this text classification method in the next lab.)

To perform nearest neighbor classification, we need a method for measuring the (metaphorical) distance between two texts based on their representations. We'll explore a few methods here:

* Hamming distance
* Jaccard distance
* Euclidean distance
* cosine distance

You'll implement code for all of these distance metrics. Try to implement the functions using `torch` tensor functions only, without explicit iteration over the elements in the vector.

We'll take a look at the distances among the following sentences:

1. Would you like them here or there?
2. I would not like them here or there.
3. Do you like green eggs and ham?
4. I do not like them Sam-I-am.

We'll start with the set of words representations of these sentences:

In [43]:
examples = """Would you like them here or there?
              I would not like them here or there.
              Do you like green eggs and ham?
              I do not like them Sam-I-am.""" \
           .split("\n")
sows = [set_of_words(nltk_normpunc_tokenize(sentence), vocabulary)
            for sentence in examples]

In [44]:
sows

[tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
         1.]),
 tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
         0.]),
 tensor([0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         1.]),
 tensor([0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0.,
         0.])]

## Hamming distance

The Hamming distance between two vectors is the number of positions at which they differ. Define a function `hamming_distance` that computes the Hamming distance between two vectors. The returned value should be a (tensorized) integer.
<!--
BEGIN QUESTION
name: hamming_distance
-->

In [45]:
#TODO
def hamming_distance(v1, v2):
    return torch.sum(v1 != v2)

# v1 = torch.tensor([1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
# v2 = torch.tensor([0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
# print(hamming_distance(v1, v2))

In [46]:
grader.check("hamming_distance")

All tests passed!

Now we can generate the Hamming distances among all of the sample sentences in a little table. Do the values make sense? What does the diagonal tell you?

In [47]:
for i in range(4):
    for j in range(4):
        print(f"{hamming_distance(sows[i], sows[j]):4} ", end='')
    print()

   0    5   10   11 
   5    0   15    6 
  10   15    0   11 
  11    6   11    0 


## Jaccard distance

The Jaccard distance between two sets (and remember that these bit strings basically represent sets) is one minus the number of elements in their intersection divided by the number of elements in their union.

$$ D_{jaccard}(v_1, v_2) = 1 - \frac{| v_1 \cap v_2 |}{| v_1 \cup v_2 |} $$

Define a function `jaccard_distance` to compute the Jaccard distance between two set-of-words representations. The returned value should be a (tensorized) float.
<!--
BEGIN QUESTION
name: jaccard_distance
-->

In [48]:
#TODO
def jaccard_distance(v1, v2):
    v1_int = v1.type(torch.int)
    v2_int = v2.type(torch.int)
    intersection = torch.sum(v1_int&v2_int)
    union = torch.sum(v1_int|v2_int)
    return 1 - (intersection / union)

In [49]:
grader.check("jaccard_distance")

All tests passed!

Again, here's a table of the Jaccard distances among the sample sentences.

In [50]:
for i in range(4):
    for j in range(4):
        print(f"{jaccard_distance(sows[i], sows[j]):5.3f} ", end='')
    print()

0.000 0.455 0.769 0.846 
0.455 0.000 0.938 0.545 
0.769 0.938 0.000 0.846 
0.846 0.545 0.846 0.000 


## Euclidean distance

The Euclidean distance between two vectors is the norm of the vector between them, that is,

$$ D_{euclidean}(\mathbf{x}, \mathbf{y}) = |\mathbf{x} - \mathbf{y}| $$

where $|\mathbf{z}|$, the norm of a vector $\mathbf{z}$, is calculated as

$$ |\mathbf{z}| = \sqrt{\sum_{i=1}^N \mathbf{z}_i^2} $$

Fortunately, `torch` provides the function [`torch.linalg.norm`](https://pytorch.org/docs/stable/generated/torch.linalg.norm.html#torch.linalg.norm) to compute the norm, and the vector between two vectors can be computed by componentwise subtraction.

Define a function `euclidean_distance` to compute the Euclidean distance between two vectors. (For the time being, try to implement it directly without using `torch.linalg.norm`.)
<!--
BEGIN QUESTION
name: euclidean_distance
-->

In [51]:
#TODO
def euclidean_distance(v1, v2):
    sub = v1 - v2
    return torch.sqrt(torch.sum(sub**2))

In [52]:
grader.check("euclidean_distance")

All tests passed!

Again, here's a table of the Euclidean distances among the sample sentences.

In [53]:
for i in range(4):
    for j in range(4):
        print(f"{euclidean_distance(sows[i], sows[j]):5.3f} ", end='')
    print()

0.000 2.236 3.162 3.317 
2.236 0.000 3.873 2.449 
3.162 3.873 0.000 3.317 
3.317 2.449 3.317 0.000 


## Cosine distance

The _cosine similarity_ of two vectors of length $N$ is the cosine of the angle that they form, which is computed as the dot product of the two vectors divided by their norms.

$$ cos(\mathbf{x}, \mathbf{y}) =
      \frac{\sum_{i=1}^N \mathbf{x}_i \cdot \mathbf{y}_i}{|\mathbf{x}| \cdot |\mathbf{y}|} $$

This isn't a distance metric, but a similarity metric. For vectors of non-negative numbers, it ranges from 0 to 1, where 0 is maximally different and 1 is maximally similar. To turn it into a distance metric, then, we take the inverse cosine (to convert the cosine to an angle between $\pi$ and 0) and divide by $\pi$.

$$ D_{cosine}(\mathbf{x}, \mathbf{y}) = \frac{cos^{-1}(cos(\mathbf{x}, \mathbf{y}))}{\pi} $$

Since we're using `torch`, some of these functions are already provided. See hints [here](https://pytorch.org/docs/stable/generated/torch.dot.html), [here](https://pytorch.org/docs/stable/generated/torch.linalg.norm.html), and [here](https://docs.python.org/3/library/math.html#math.acos).

(To avoid some math domain errors, we recommend that you use the function `safe_acos` that we've provided to compute the inverse cosine function instead of using `math.acos` directly.)

In [54]:
def safe_acos(x):
    """Returns the arc cosine of `x`. Unlike `math.acos`, it
       does not raise an exception for values of `x` out of range,
       but rather clips `x` at -1..1, thereby avoiding math domain
       errors in the case of numerical errors."""
    return math.acos(math.copysign(min(1.0, abs(x)), x))

In [55]:
#TODO
def cosine_distance(v1, v2):
    """Returns the cosine distance between two vectors"""
    dot_product = torch.dot(v1, v2)
    norm_v1 = torch.linalg.norm(v1)
    norm_v2 = torch.linalg.norm(v2)
    return safe_acos(dot_product / (norm_v1 * norm_v2)) / math.pi

In [56]:
grader.check("cosine_distance")

All tests passed!

In [57]:
for i in range(4):
    for j in range(4):
        print(f"{cosine_distance(sows[i], sows[j]):5.3f} ", end='')
    print()

0.000 0.250 0.378 0.414 
0.250 0.000 0.462 0.283 
0.378 0.462 0.000 0.414 
0.414 0.283 0.414 0.000 


In the next lab, you'll use some of these distance metrics to automatically classify text using nearest neighbor classification.

<!-- BEGIN QUESTION -->

# Lab debrief

**Question:** We're interested in any thoughts you have about this lab so that we can improve this lab for later years, and to inform later labs for this year. Please list any issues that arose or comments you have to improve the lab. Useful things to comment on include the following, but you're not restricted to these:

* Was the lab too long or too short?
* Were the readings appropriate for the lab?
* Was it clear (at least after you completed the lab) what the points of the exercises were?
* Are there additions or changes you think would make the lab better?

<!--
BEGIN QUESTION
name: open_response_debrief
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



# Submission Instructions

This lab should be submitted to Gradescope at <https://www.gradescope.com/courses/903849>, under "lab 1-1 - Code", which will be made available some time before the due date.

Make sure that you have passed all public tests by running `grader.check_all()` below before submitting. Note that there are hidden tests on Gradescope, the results of which will be revealed after the submission deadline.

We also request that you **submit a PDF of the freshly run notebook**. For that, the best method is to press "restart kernel and run all cells", allowing time for all cells to be run to completion. Then, export the notebook as a PDF, and submit it to Gradescope, under the assignment "lab1-1 - PDF".

# End of lab 1-1

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [58]:
grader.check_all()

anywhere_1hot:

    All tests passed!
    

anywhere_1hot_reverse:

    All tests passed!
    

cosine_distance:

    All tests passed!
    

euclidean_distance:

    All tests passed!
    

example_bow:

    All tests passed!
    

example_sow:

    All tests passed!
    

hamming_distance:

    All tests passed!
    

jaccard_distance:

    All tests passed!
    

nltk_text_tokenized:

    All tests passed!
    

nltk_whitespace_tokenize_and_nltk_normpunc_tokenize:

    All tests passed!
    

normalize_token:

    All tests passed!
    

text_tokenized:

    All tests passed!
    

token_count:

    All tests passed!
    

token_count_punc:

    All tests passed!
    

token_count_whitespace:

    All tests passed!
    

tokens_whitespace:

    All tests passed!
    

type_count:

    All tests passed!
    

type_count_norm_punc:

    All tests passed!